# Inventory Management Environments

> To be written.



In [ ]:
#| default_exp agents.newsvendor.erm

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import logging

from abc import ABC, abstractmethod
from typing import Union, Optional, List
import numpy as np
import os


from ddopnew.envs.base import BaseEnvironment
from ddopnew.agents.base import BaseAgent
from ddopnew.utils import MDPInfo, Parameter, DatasetWrapper
from ddopnew.torch_utils.loss_functions import TorchQuantileLoss
from ddopnew.torch_utils.preprocessors import FlattenTimeDim

from ddopnew.dataloaders.base import BaseDataLoader

import torch

In [ ]:
#| export

class SGDBaseAgent(BaseAgent):

    train_mode = "epochs_fit"
    
    def __init__(self, 
            environment_info: MDPInfo,
            dataloader: BaseDataLoader,
            optimizer_params: Optional[dict] = None,  # default: {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}
            learning_rate_scheduler = None,  # TODO: add base class for learning rate scheduler for typing
            dataloader_params: Optional[dict] = None, # default: {"batch_size": 32, "shuffle": True}
            preprocessors: Optional[List] = None,     # default: []
            postprocessors: Optional[List] = None,     # default: []
            torch_preprocessors: Optional[List] = None,     # default: []
            device: str = "cpu" # "cuda" or "cpu"
            ):

        # Initialize default values for mutable arguments
        optimizer_params = optimizer_params or {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}
        dataloader_params = dataloader_params or {"batch_size": 32, "shuffle": True}
        self.torch_preprocessors = torch_preprocessors or []

        self.device = device
        
        self.set_dataloader(dataloader, dataloader_params)
        self.set_model()
        self.set_loss_function()
        self.set_optimizer(optimizer_params)
        self.set_learning_rate_scheduler(learning_rate_scheduler)

        super().__init__(environment_info, preprocessors, postprocessors)

    def set_dataloader(self, dataloader, dataloader_params):
        dataset = DatasetWrapper(dataloader)
        self.dataloader = torch.utils.data.DataLoader(dataset, **dataloader_params)

    @abstractmethod
    def set_loss_function(self):
        pass

    @abstractmethod
    def set_model(self):
        pass

    def set_optimizer(self, optimizer_params):
        optimizer = optimizer_params["optimizer"]
        optimizer_params_copy = optimizer_params.copy()
        del optimizer_params_copy["optimizer"]

        if optimizer == "Adam":
            self.optimizer = torch.optim.Adam(self.model.parameters(), **optimizer_params_copy)
        elif optimizer == "SGD":
            self.optimizer = torch.optim.SGD(self.model.parameters(), **optimizer_params_copy)
        elif optimizer == "RMSprop":
            self.optimizer = torch.optim.RMSprop(self.model.parameters(), **optimizer_params_copy)
        else:
            raise ValueError(f"Optimizer {optimizer} not supported")
        
    def set_learning_rate_scheduler(self, learning_rate_scheduler):
        if learning_rate_scheduler is not None:
            raise NotImplementedError("Learning rate scheduler not implemented yet")
        else:
            self.learning_rate_scheduler = None

    def fit_epoch(self):

        device = next(self.model.parameters()).device
        self.model.train()
        total_loss=0

        for i, output in enumerate(self.dataloader):
            
            X, y = output

            # convert X and y to float32
            X = X.type(torch.float32)
            y = y.type(torch.float32)

            for torch_preprocessor in self.torch_preprocessors:
                X = torch_preprocessor(X)
            
            X, y = X.to(device), y.to(device)

            self.optimizer.zero_grad()

            y_pred = self.model(X)

            if self.loss_function_params==None:
                loss = self.loss_function(y_pred, y)
            else:
                loss = self.loss_function(y_pred, y, **self.loss_function_params) # TODO: add reduction param when defining loss function

            loss.backward()
            self.optimizer.step()
        
            total_loss += loss.item()
        
        self.model.eval()
        
        return total_loss

    def draw_action_(self, observation):
        
        action = self.predict(observation)
        
        return action

    def predict(self, X):

        # TODO handle if X is larger than some size, then split into batches

        device = next(self.model.parameters()).device
        self.model.eval()

        X = torch.tensor(X, dtype=torch.float32)
        for torch_preprocessor in self.torch_preprocessors:
            X = torch_preprocessor(X)
        X = X.to(device)

        with torch.no_grad():

            y_pred = self.model(X)


        y_pred = y_pred.cpu().numpy()

        return y_pred

    def train(self):
        self.model.train()

    def eval(self):
        self.model.eval()

    def to(self, device):
        self.model.to(device)

    @staticmethod
    def update_model_params(default_params, custom_params):
        updated_params = default_params.copy()
        updated_params.update(custom_params)
        return updated_params

    def save(self, path: str, overwrite=True):
        """
        Save the PyTorch model to a file in the specified directory.

        Parameters:
        - path (str): The directory where the model file will be saved.
        - overwrite (bool): If True, the file will be overwritten if it already exists. 
                            If False, a FileExistsError will be raised if the file exists.

        Raises:
        - AttributeError: If the model attribute is not set.
        - FileExistsError: If the file already exists and overwrite is set to False.
        """
        
        if not hasattr(self, 'model') or self.model is None:
            raise AttributeError("Model is not defined in the class.")

        # Create the directory path if it does not exist
        os.makedirs(path, exist_ok=True)

        # Construct the file path using os.path.join for better cross-platform compatibility
        full_path = os.path.join(path, "model.pth")

        if os.path.exists(full_path):
            if not overwrite:
                raise FileExistsError(f"The file {full_path} already exists and will not be overwritten.")
            else:
                logging.info(f"Overwriting file {full_path}") # Only log with info as during training we will continuously overwrite the model
        
        # Save the model's state_dict using torch.save
        torch.save(self.model.state_dict(), full_path)
        logging.info(f"Model saved successfully to {full_path}")

    def load(self, path: str):
        """
        Load the PyTorch model from a file.

        Parameters:
        - path (str): The directory where the model file is located.

        Raises:
        - FileNotFoundError: If the file does not exist.
        - AttributeError: If the model attribute is not set.
        - RuntimeError: If the model state cannot be loaded due to shape or parameter mismatch.
        """
        
        if not hasattr(self, 'model') or self.model is None:
            raise AttributeError("Model is not defined in the class.")

        # Construct the file path
        full_path = os.path.join(path, "model.pth")

        if not os.path.exists(full_path):
            raise FileNotFoundError(f"The file {full_path} does not exist.")

        try:
            # Load the model's state_dict using torch.load
            self.model.load_state_dict(torch.load(full_path))
            logging.info(f"Model loaded successfully from {full_path}")
        except Exception as e:
            raise RuntimeError(f"An error occurred while loading the model: {e}")
    

In [ ]:
#| export

class NVBaseAgent(SGDBaseAgent):

    def __init__(self, 
        environment_info: MDPInfo,
        dataloader: BaseDataLoader,
        cu: Union[np.ndarray, Parameter],
        co: Union[np.ndarray, Parameter],
        optimizer_params: Optional[dict] = None,  # default: {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}
        learning_rate_scheduler = None,  # TODO: add base class for learning rate scheduler for typing
        dataloader_params: Optional[dict] = None, # default: {"batch_size": 32, "shuffle": True}
        preprocessors: Optional[List] = None,     # default: []
        postprocessors: Optional[List] = None,     # default: []
        torch_preprocessors: Optional[List] = None,     # default: []
        device: str = "cpu"  # "cuda" or "cpu"
        ):
        
        self.sl = cu / (cu + co) # ensure this works if cu and co are Parameters

        super().__init__(environment_info, dataloader, optimizer_params, learning_rate_scheduler, dataloader_params, preprocessors, postprocessors,torch_preprocessors, device)


    def set_loss_function(self):

        self.loss_function_params = {"quantile": self.sl}
        self.loss_function = TorchQuantileLoss(reduction="mean")
        
        logging.debug(f"Loss function set to {self.loss_function}")

In [ ]:
#| export

class NewsvendorlERMAgent(NVBaseAgent):

    def __init__(self, 
        environment_info: MDPInfo,
        dataloader: BaseDataLoader,
        cu: Union[np.ndarray, Parameter],
        co: Union[np.ndarray, Parameter],
        optimizer_params: Optional[dict] = None,  # default: {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}
        learning_rate_scheduler = None,  # TODO: add base class for learning rate scheduler for typing
        model_params: Optional[dict] = None,      # default: {"input_size": 1, "output_size": 1, "relu_output": False}
        dataloader_params: Optional[dict] = None, # default: {"batch_size": 32, "shuffle": True}
        preprocessors: Optional[List] = None,     # default: 
        postprocessors: Optional[List] = None,     # default: []
        torch_preprocessors: Optional[List] = None,     # default: [FlattenTimeDim(allow_2d=False)]
        device: str = "cpu"  # "cuda" or "cpu"
        ):


        # Handle mutable defaults unique to this class
        default_model_params = {
            "input_size": 1,
            "output_size": 1,
            "relu_output": False
            }

        self.model_params = self.update_model_params(default_model_params, model_params or {})

        torch_preprocessors = [FlattenTimeDim(allow_2d=True)] if torch_preprocessors is None else torch_preprocessors

        super().__init__(environment_info, dataloader, cu, co, optimizer_params, learning_rate_scheduler, dataloader_params, preprocessors, postprocessors, torch_preprocessors, device)
    
    def set_model(self):
        from ddopnew.approximators import LinearModel
        self.model = LinearModel(**self.model_params)

In [ ]:
from ddopnew.envs.inventory import NewsvendorEnv
from ddopnew.dataloaders.tabular import XYDataLoader
from ddopnew.experiment_functions import run_experiment, test_agent

In [ ]:
val_index_start = 800 #90_000
test_index_start = 900 #100_000

X = np.random.rand(1000, 2)
Y = np.random.rand(1000, 1)

dataloader = XYDataLoader(X, Y, val_index_start, test_index_start)

environment = NewsvendorEnv(
    dataloader = dataloader,
    underage_cost = 0.42857,
    overage_cost = 1.0,
    gamma = 0.999,
    horizon_train = 365,
)

agent = NewsvendorlERMAgent(environment.mdp_info,
                            dataloader,
                            cu=np.array([0.42857]),
                            co=np.array([1.0]),
                            optimizer_params= {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}, # other optimizers: "SGD", "RMSprop"
                            learning_rate_scheduler = None, # TODO add base class for learning rate scheduler for typing
                            model_params = {"input_size": 2, "output_size": 1, "relu_output": False}, #
                            dataloader_params={"batch_size": 32, "shuffle": True},
                            torch_preprocessors = [],
                            device = "cpu" # "cuda" or "cpu"
)

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

run_experiment(agent, environment, 2, run_id = "test") # fit agent via run_experiment function

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

-18.051547487461786 -17.19908573287191
-14.24499509513067 -13.575595456292424


In [ ]:
#| export

class NewsvendorDLAgent(NVBaseAgent):

    def __init__(self, 
        environment_info: MDPInfo,
        dataloader: BaseDataLoader,
        cu: Union[np.ndarray, Parameter],
        co: Union[np.ndarray, Parameter],
        optimizer_params: Optional[dict] = None,  # default: {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}
        learning_rate_scheduler = None,  # TODO: add base class for learning rate scheduler for typing
        model_params: Optional[dict] = None,      # default: {"input_size": 1, "output_size": 1, "relu_output": False}
        dataloader_params: Optional[dict] = None, # default: {"batch_size": 32, "shuffle": True}
        preprocessors: Optional[List] = None,     # default: 
        postprocessors: Optional[List] = None,     # default: []
        torch_preprocessors: Optional[List] = None,     # default: [FlattenTimeDim(allow_2d=False)]
        device: str = "cpu"  # "cuda" or "cpu"
        ):

        # Handle mutable defaults unique to this class
        default_model_params = {
            "input_size": 1,
            "output_size": 1,
            "hidden_layers": [64, 64],
            "drop_prob": 0.0,
            "batch_norm": False,
            "relu_output": False
            }

        self.model_params = self.update_model_params(default_model_params, model_params or {})
        
        torch_preprocessors = [FlattenTimeDim(allow_2d=True)] if torch_preprocessors is None else torch_preprocessors

        super().__init__(environment_info, dataloader, cu, co, optimizer_params, learning_rate_scheduler, dataloader_params, preprocessors, postprocessors, torch_preprocessors, device)
    
    def set_model(self):
        from ddopnew.approximators import MLP
        self.model = MLP(**self.model_params)

In [ ]:
dataloader = XYDataLoader(X, Y, val_index_start, test_index_start)

environment = NewsvendorEnv(
    dataloader = dataloader,
    underage_cost = 0.42857,
    overage_cost = 1.0,
    gamma = 0.999,
    horizon_train = 365,
)


model_params = {
    "input_size": 2,
}

agent = NewsvendorDLAgent(environment.mdp_info,
                            dataloader,
                            cu=np.array([0.42857]),
                            co=np.array([1.0]),
                            optimizer_params= {"optimizer": "Adam", "lr": 0.01, "weight_decay": 0.0}, # other optimizers: "SGD", "RMSprop"
                            learning_rate_scheduler = None, # TODO add base class for learning rate scheduler for typing
                            model_params = model_params, #
                            dataloader_params={"batch_size": 32, "shuffle": True},
                            torch_preprocessors = [],
                            device = "cpu" # "cuda" or "cpu"
)

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

run_experiment(agent, environment, 2, run_id = "test") # fit agent via run_experiment function

environment.test()
agent.eval()

R, J = test_agent(agent, environment)

print(R, J)

-18.911026013892176 -17.999486975878526
-13.953989446869352 -13.277391185500251


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()